In [4]:
%load_ext autoreload
%autoreload 2
import torch
import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from misc.cross_frame_attn import CrossFrameAttnProcessor

device = torch.device('cuda')
torch_dtype = torch.float16
sd_repo = "runwayml/stable-diffusion-v1-5"
controlnet_repo = "lllyasviel/sd-controlnet-depth"

controlnet = ControlNetModel.from_pretrained(
    controlnet_repo,
    torch_dtype=torch_dtype,
    device=device
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_repo,
    controlnet=controlnet,
    device=device,
    torch_dtype=torch_dtype,
    safety_checker=None
)
pipe.enable_model_cpu_offload(gpu_id=0)

cross_frame_processor = CrossFrameAttnProcessor(unet_chunk_size=2)
pipe.unet.set_attn_processor(processor=cross_frame_processor)
pipe.controlnet.set_attn_processor(processor=cross_frame_processor)

/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Keyword arguments {'device': device(type='cuda')} are not expected by StableDiffusionControlNetPipeline and will be ignored.
Loading pipeline components...:  67%|██████▋   | 4/6 [00:00<00:00,  4

In [3]:
from text3d2video.artifacts.animation_artifact import AnimationArtifact
from text3d2video.util import front_camera
import wandb

api = wandb.Api()
animation = 'backflip:latest'
animation = api.artifact(f'romeu/diffusion-3D-features/{animation}')
animation = AnimationArtifact.from_wandb_artifact(animation)

batch_size = 7
frames = animation.load_frames(range(1, batch_size + 1))
camera = front_camera()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Downloading large artifact backflip:latest, 121.09MB. 61 files... 
wandb:   61 of 61 files downloaded.  
Done. 0:0:0.4
/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [4]:
from text3d2video.rendering import make_rasterizer, normalize_depth_map
import torchvision.transforms.functional as TF

rasterizer = make_rasterizer(camera, 512)
fragments = rasterizer(frames)
depth_map_normalized = normalize_depth_map(fragments.zbuf)
depth_ims = [TF.to_pil_image(depth_map_normalized[i, :, :, 0].cpu()) for i in range(batch_size)]

In [34]:
from scripts.generate_video import generate_video
from text3d2video.artifacts.video_artifact import VideoArtifact
from text3d2video.ipython_utils import display_ims

wandb.init(project='diffusion-3D-features', job_type='generate_video')

name = 'spiderman'
prompt = "Spiderman doing a backflip"

wandb.log({'prompt': prompt})
wandb.use_artifact(animation.wandb_artifact)

ims = generate_video(pipe, prompt, depth_ims)

video = VideoArtifact.create_wandb_artifact(f'{name}-cross-frame', frames=ims, fps=30)
wandb.log_artifact(video)

prompt,Stormtrooper doing a...


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


Moviepy - Building video /tmp/tmpa71hikgk/video.mp4.
Moviepy - Writing video /tmp/tmpa71hikgk/video.mp4



wandb: Adding directory to artifact (/tmp/tmpa71hikgk)... Done. 0.0s


Moviepy - Done !
Moviepy - video ready /tmp/tmpa71hikgk/video.mp4


<Artifact spiderman-cross-frame>

In [24]:
from text3d2video.artifacts.video_artifact import VideoArtifact
from text3d2video.wandb_util import api_artifact

artifact_tag = 'stormtrooper-cross-frame:latest'

video = api_artifact(artifact_tag)
video = VideoArtifact.from_wandb_artifact(video)
video.ipy_display()

wandb:   1 of 1 files downloaded.  
